# Projeto Final: Módulo de Programação Orientada a Objetos

Foi criado um programa para extrair dados de uma API. A API escolhida foi a Brasil API.



### **Integrantes**:<a name="integrantes"></a>

>
- Beatriz Porto
- Eduardo Mattana
- Michel Camargo
- Pedro Lima
- Rubens Sousa

### Setup

In [ ]:
%pip install requests


In [ ]:
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
import requests # type: ignore
import json
from functools import reduce
import sqlalchemy # type: ignore
print('sqlalchemy.__version__:', sqlalchemy.__version__)
import sqlite3
from sqlalchemy import create_engine # type: ignore
import sys

sqlalchemy.__version__: 2.0.32


### Classes

In [ ]:
class CriaRequisicoes:
    def __init__(self,valor=""):
        self.link = "https://brasilapi.com.br/api/"
        self.valor = valor

    def retorna_infos_cep(self):
      return self.link + "cep" + "/" + "v1" + "/" + self.valor

    def retorna_infos_cnpj(self):
      return self.link + "cnpj" + "/" + "v1" + "/" + self.valor

    def retorna_infos_corretora(self):
      return self.link + "cvm" + "/" + "corretoras" + "/" + "v1" + "/" + self.valor

    def retorna_lista_corretora(self):
      return self.link + "cvm" + "/" + "corretoras" + "/" + "v1"

    def retorna_lista_localidades(self):
      return self.link + "cptec" + "/" + "v1" + "/" + "cidade"


    def retorna_busca_localidades(self):
      return self.link + "cptec" + "/" + "v1" + "/" + "cidade" + "/" + self.valor

    def retorna_lista_condicoes_metereologicas_capitais(self):
      return self.link + "cptec" + "/" + "v1" + "/" + "clima" + "/" + "capital"


    def retorna_busca_condicoes_metereologicas_cidade(self):
      return self.link + "cptec" + "/" + "v1" + "/" + "clima" + "/" + "previsao" + "/" + self.valor


    def retorna_lista_cidades_por_DDD(self):
      return self.link + "ddd" + "/" + "v1" + "/" +  self.valor

    def retorna_lista_ncm(self):
      return self.link + "ncm" + "/" + "v1"


    def retorna_lista_ncm(self):
      return self.link + "ncm" + "/" + "v1"


    def retorna_livro_por_isbn(self):
      return self.link + "isbn" + "/" + "v1" + "/" + self.valor






In [ ]:
class RealizaRequisicao:
  def __init__(self,url):
       self.url = url

  def retorna_dados(self):
    response = requests.get(self.url)
    if response.status_code == 200:

      data = response.json()
      return data
    else:
      print(f"Erro na requisição: {response.status_code}")

In [ ]:
class ConsultaCNPJ:
    def __init__(self, cnpj):
        self.cnpj = cnpj
        self.url =  CriaRequisicoes(cnpj).retorna_infos_cnpj()
        self.dados_cnpj = None

    def buscar_dados(self):
        try:
          dados = RealizaRequisicao(self.url).retorna_dados()

          self.dados_cnpj = dados


        except Exception as err:
          print('CNPJ não localizado ou inexistete.')


    def buscar_socios(self):
        if self.dados_cnpj is None:
            self.buscar_dados()

        df_dados = pd.json_normalize(self.dados_cnpj)
        df_exploded = df_dados.explode('qsa')
        df_qsa = pd.json_normalize(df_exploded['qsa'])

        print('SOCIOS DA EMPRESA')
        print('-'*150)

        for i, df in df_qsa.iterrows():
            print(f"Nome socio {i+1}: {df['nome_socio']: <35} | Faixa etária: {df['faixa_etaria']: <25} | Cargo: {df['qualificacao_socio']: <15} | Entrada na sociedade: {df['data_entrada_sociedade']}")

        print('-'*150)

    def buscar_cnaes(self):
        if self.dados_cnpj is None:
            self.buscar_dados()

        df_dados = pd.json_normalize(self.dados_cnpj)
        df_exploded = df_dados.explode('cnaes_secundarios')
        df_cnaes = pd.json_normalize(df_exploded['cnaes_secundarios'])

        print('CNAES DA EMPRESA')
        print('-'*150)

        for i, df in df_cnaes.iterrows():
            print(f"Cod. Cnae {i+1}: {df['codigo']: <10} | Descrição: {df['descricao']}")

        print('-'*150)

In [ ]:
class ConsultaCEP:
    def __init__(self, cep):
        self.cep = cep
        self.url =  CriaRequisicoes(cep).retorna_infos_cep()
        self.dados_cep = None

    def buscar_dados(self):
        try:
          dados = RealizaRequisicao(self.url).retorna_dados()
          self.dados_cep = dados

        except Exception as err:
          print('CEP não localizado ou inexistete:')
          print(f'{err}')

    def informacoes_cep(self):
      if self.dados_cep is None:
            self.buscar_dados()
      df_dados = pd.json_normalize(self.dados_cep)
      print("DADOS DO LOGRADOURO")
      print('-'*150)
      for i, df in df_dados.iterrows():
        print(f"CEP:{str(df['cep']): <15} | Estado: {str(df['state']): <10} | Cidade: {str(df['city']): <15} | Complemento: {str(df['neighborhood']): <15} | Rua: {str(df['street']): <35} | Serviço: {str(df['service'])}")




In [ ]:
class ConsultaCorretora:
  def __init__(self, cnpj):
        self.cnpj = cnpj
        self.url =  CriaRequisicoes(cnpj).retorna_infos_corretora()
        self.dados_corretora = None
  def buscar_dados(self):
    try:
      dados = RealizaRequisicao(self.url).retorna_dados()
      self.dados_corretora = dados

    except Exception as err:
      print('CNPJ não localizado ou inexistete:')
      print(f'{err}')

  def informacoes_logradouro(self):
      if self.dados_corretora is None:
            self.buscar_dados()
      df_dados = pd.json_normalize(self.dados_corretora)
      print("DADOS DE LOGRADOURO DA CORRETORA")
      print('-'*150)
      for i, df in df_dados.iterrows():
        print(f"CNPJ:{str(df['cnpj']): <15} | Estado: {str(df['uf']): <10} | Cidade: {str(df['municipio']): <15} | Bairro: {str(df['bairro']): <15} | Logradouro: {str(df['logradouro']): <35} | Complemento: {str(df['complemento'])}")

  def informacoes_cadastro(self):
      if self.dados_corretora is None:
            self.buscar_dados()
      df_dados = pd.json_normalize(self.dados_corretora)
      print("DADOS CADASTRAIS DA CORRETORA")
      print('-'*150)
      for i, df in df_dados.iterrows():
        print(f"CNPJ:{str(df['cnpj']): <15} | Razão Social: {str(df['nome_social']): <15} | Nome Fantasia: {str(df['nome_comercial']): <15} | Status: {str(df['status']): <15} | Email: {str(df['email']): <35} | Telefone: {str(df['telefone'])}")

  def informacoes_patrimonio(self):
      if self.dados_corretora is None:
            self.buscar_dados()
      df_dados = pd.json_normalize(self.dados_corretora)
      print("DADOS FINANCERIOS DA CORRETORA")
      print('-'*150)
      for i, df in df_dados.iterrows():
        print(f"CNPJ:{str(df['cnpj']): <15} | Data Patrimonio Liquido: {str(df['data_patrimonio_liquido']): <15} | Valor patrimonio liquido: {str(df['valor_patrimonio_liquido'])}")



In [ ]:
class ConsultaCidadesMatereologia:
  def __init__(self, nome):
        self.nome = nome
        self.url =  CriaRequisicoes(nome).retorna_busca_localidades()
        self.dados_cidade = None
  def buscar_dados(self):
    try:
      dados = RealizaRequisicao(self.url).retorna_dados()
      self.dados_cidade = dados

    except Exception as err:
      print('Nome não localizado ou inexistete:')
      print(f'{err}')

  def informacoes_cidade(self):
      if self.dados_cidade is None:
            self.buscar_dados()
      df_dados = pd.json_normalize(self.dados_cidade)
      print("DADOS DE CIDADE")
      print('-'*150)
      for i, df in df_dados.iterrows():
        print(f"ID:{str(df['id']): <15} | Estado: {str(df['estado']): <10} | Cidade: {str(df['nome'])}")

  def informacoes_metereologicas_cidade(self):
      if self.dados_cidade is None:
            self.buscar_dados()


      df_dados = pd.json_normalize(self.dados_cidade)

      url_lista =["https://brasilapi.com.br/api/cptec/v1/clima/previsao/" + "/" + str(a) for a in df_dados["id"]]

      dados_coletados = []
      for url in url_lista:
        dados_meterologicos = RealizaRequisicao(url).retorna_dados()
        dados_coletados.append(dados_meterologicos)
      df_dados_clima = pd.json_normalize(dados_coletados)
      df_exploded = df_dados_clima.explode('clima')
      df_dados_clima = pd.json_normalize(df_exploded['clima'])
      df_final = pd.concat([df_exploded.drop(columns='clima'), df_dados_clima], axis=1)

      print("DADOS METEOROLÓGICOS")
      print('-'*150)
      for i, df in df_final.iterrows():
        print(f"CIDADE:{str(df['cidade']): <15} | Estado: {str(df['estado']): <15} | Data Última Atualização: {str(df['atualizado_em']): <15} | Descrição da condição: {str(df['condicao_desc']): <15} | Mínima:{str(df['min']): <15} | Máxima:{str(df['max']): <15} | Indice UV :{str(df['indice_uv']): <15}")

In [ ]:
class DataBaseSQLite3:

    def __init__(self, db_name):

       self.db_name = db_name

    def create_database(self):
        ''' Método para criar uma base de dados local em SQLite3'''

        try:
          conn = sqlite3.connect(self.db_name)
          print(f"Database {self.db_name} formed.")
          cursor = conn.cursor()
          print(dir(cursor))
        except sqlite3.Error as error:
          print(f"Database {self.db_name} not formed.")
          print(f"Failed to connect with {self.db_name} database", error)

    def send_dataframe_to_database(
                    self,
                    dataframe,
                    table_name = '',
                    ):

            db = self.db_name
            db_engine = create_engine(f'sqlite:///{db}')


            dataframe.to_sql(
                name = table_name,
                con = db_engine,
                if_exists = 'replace',
                index = False
                )

            connection = sqlite3.connect(self.db_name)

            print(connection.total_changes)

    def check_tables_in_database(self):

        query = '''
        SELECT
            name
        FROM
            sqlite_schema
        WHERE
            type ='table' AND
            name NOT LIKE 'sqlite_%';
        '''

        connection = sqlite3.connect(self.db_name)
        print('connection.total_changes:', connection.total_changes)

        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()

        tables = cursor.execute(query).fetchall()


        print(f'tables in database {self.db_name}:', tables)

    def read_table_in_database(
            self,
            table_name = '',
            n_rows = 20
            ):

        query = f'''
        SELECT
            *
        FROM
            {table_name}
        '''

        connection = sqlite3.connect(self.db_name)
        cursor = connection.cursor()
        table = cursor.execute(query).fetchmany(n_rows)
        cursor.close()

        return table

    def delete_database(self):

        if os.path.exists(self.db_name):
            os.remove(self.db_name)
            print("Successfully! The File has been removed")
        else:
            print("Can not delete the file as it doesn't exists")


    def export_schema(self, file_name):
        conn = sqlite3.connect(self.db_name)
        with open(file_name, 'w') as f:
            for line in conn.iterdump():
                f.write('%s\n' % line)
        print(f"Database schema exported to {file_name}")

    def execute_query(self, query):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        try:
            cursor.execute(query)
            conn.commit()
            return cursor.fetchall()
        except sqlite3.Error as error:
            print(f"Error executing query: {error}")
        finally:
            cursor.close()


In [ ]:
class GerenciarDados:
    def __init__(self, db_name,valor):
        self.name = db_name
        self.db = DataBaseSQLite3(db_name)
        self.valor = valor
        self.link = "https://brasilapi.com.br/api/"
        self.dados = None
    def buscar_dados(self):
      try:
        if self.valor == 'corretoras':
          url = self.link + "cvm" + "/" + "corretoras" + "/" + "v1"
          dados = RealizaRequisicao(url).retorna_dados()
          self.dados = dados

        elif self.valor == 'cidade':
          url = self.link +  "cptec" + "/" + "v1" + "/" + "cidade"
          dados = RealizaRequisicao(url).retorna_dados()
          self.dados = dados

        elif self.valor == 'clima':
          url = self.link +  "cptec" + "/" + "v1" + "/" + "clima" + "/" + "capital"
          dados = RealizaRequisicao(url).retorna_dados()
          self.dados = dados
        elif self.valor=='ncm':
          url = self.link + "ncm" + "/" + "v1"
          dados = RealizaRequisicao(url).retorna_dados()
          self.dados = dados

      except Exception as err:
        print('Informação não localizada ou inexistete:')
        print(f'{err}')

    def processar_dados(self):
      if self.dados is None:
              self.buscar_dados()
      df_final = pd.DataFrame()
      if self.valor == 'corretoras':
          df_final = pd.json_normalize(self.dados)
      elif self.valor == 'cidade':
         df_final = pd.json_normalize(self.dados)

      elif self.valor == 'clima':
        df_final = pd.json_normalize(self.dados)

      elif self.valor=='ncm':
        df_final = pd.json_normalize(self.dados)

      return df_final


### Criação de Banco de Dados Local

In [ ]:
banco = DataBaseSQLite3("dados_gerais")
banco.create_database()
cidades = GerenciarDados("dados_gerais","cidade")
corretoras = GerenciarDados("dados_gerais","corretoras")
#clima = GerenciarDados("dados_gerais","clima")
ncm = GerenciarDados("dados_gerais","ncm")


Database dados_gerais formed.
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'close', 'connection', 'description', 'execute', 'executemany', 'executescript', 'fetchall', 'fetchmany', 'fetchone', 'lastrowid', 'row_factory', 'rowcount', 'setinputsizes', 'setoutputsize']


In [ ]:
cidades.buscar_dados()
corretoras.buscar_dados()
#clima.buscar_dados()
ncm.buscar_dados()

In [ ]:
df_cidades = cidades.processar_dados()
df_corretoras = corretoras.processar_dados()
#df_clima = clima.processar_dados()
df_ncm = ncm.processar_dados()

In [ ]:
banco.send_dataframe_to_database(df_cidades,"dados_cidades")
banco.send_dataframe_to_database(df_corretoras,"dados_corretoras")
banco.send_dataframe_to_database(df_ncm,"dados_ncm")

0
0
0


### Módulos do Menu

In [ ]:
def menu_principal():
    print("\nBem-vindo ao Sistema de Consulta!")
    print("Escolha uma das opções abaixo:")
    print("1 - Consulta de Dados de Empresa (CNPJ)")
    print("2 - Consulta de CEP")
    print("3 - Consulta de Corretora (CNPJ)")
    print("4 - Consulta de Cidades e Dados Meteorológicos")
    print("5 - Gerenciamento do Banco de Dados")
    print("0 - Sair")
    return input("\nDigite a opção desejada: ")



def menu_database(db):
    while True:
        print("\nMenu do Banco de Dados:")
        print("1 - Verificar tabelas")
        print("2 - Visualizar dados de uma tabela")
        print("3 - Executar uma query")
        print("0 - Voltar")
        opcao = input("\nEscolha a opção: ")

        if opcao == '1':
            tabelas = db.check_tables_in_database()
            print("Tabelas no banco de dados:", tabelas)
        elif opcao == '2':
            table_name = input("Digite o nome da tabela: ")
            rows = db.read_table_in_database(table_name)
            for row in rows:
                print(row)
        elif opcao == '3':
            query = input("Digite a query que deseja executar: ")
            try:
                result = db.execute_query(query)
                for row in result:
                    print(row)
            except Exception as e:
                print("Query inválida! Erro:", str(e))
                retry = input("Deseja tentar novamente? (s/n): ")
                if retry.lower() != 's':
                    break
        elif opcao == '0':
            break
        else:
            print("Opção inválida!")



def menu_empresa():
    cnpj = input("\nDigite o CNPJ da empresa: ")
    empresa = ConsultaCNPJ(cnpj)

    while True:
        print("\n1 - Consultar sócios")
        print("2 - Consultar CNAEs")
        print("0 - Voltar")
        opcao = input("\nEscolha a opção: ")

        try:
            if opcao == '1':
                empresa.buscar_dados()
                empresa.buscar_socios()
            elif opcao == '2':
                empresa.buscar_dados()
                empresa.buscar_cnaes()
            elif opcao == '0':
                break
            else:
                print("Opção inválida!")
        except Exception as e:
            print(f"Ocorreu um erro: {e}")
            retry = input("Deseja tentar novamente? (s/n): ")
            if retry.lower() != 's':
                print("Saindo do menu de consulta de empresa.")
                break

def menu_cep():
    while True:
        try:
            cep = input("\nDigite o CEP: ")
            consulta_cep = ConsultaCEP(cep)
            consulta_cep.buscar_dados()
            consulta_cep.informacoes_cep()
            break
        except Exception as e:
            print(f"Ocorreu um erro: {e}")
            retry = input("Deseja tentar novamente? (s/n): ")
            if retry.lower() != 's':
                print("Saindo do menu de consulta de CEP.")
                break


def menu_corretora():
    while True:
        try:
            cnpj = input("\nDigite o CNPJ da corretora: ")
            corretora = ConsultaCorretora(cnpj)
            corretora.buscar_dados()

            while True:
                print("\n1 - Informações de Logradouro")
                print("2 - Informações de Cadastro")
                print("3 - Informações Patrimoniais")
                print("0 - Voltar")
                opcao = input("\nEscolha a opção: ")

                if opcao == '1':
                    try:
                        corretora.informacoes_logradouro()
                    except Exception as e:
                        print(f"Ocorreu um erro ao consultar logradouro: {e}")
                        retry = input("Deseja tentar novamente? (s/n): ")
                        if retry.lower() != 's':
                            break

                elif opcao == '2':
                    try:
                        corretora.informacoes_cadastro()
                    except Exception as e:
                        print(f"Ocorreu um erro ao consultar cadastro: {e}")
                        retry = input("Deseja tentar novamente? (s/n): ")
                        if retry.lower() != 's':
                            break

                elif opcao == '3':
                    try:
                        corretora.informacoes_patrimonio()
                    except Exception as e:
                        print(f"Ocorreu um erro ao consultar patrimônio: {e}")
                        retry = input("Deseja tentar novamente? (s/n): ")
                        if retry.lower() != 's':
                            break

                elif opcao == '0':
                    break
                else:
                    print("Opção inválida!")
            break
        except Exception as e:
            print(f"Ocorreu um erro ao buscar os dados da corretora: {e}")
            retry = input("Deseja tentar novamente? (s/n): ")
            if retry.lower() != 's':
                print("Saindo do menu de consulta de corretora.")
                break



def menu_cidades():
    while True:
        try:
            nome_cidade = input("\nDigite o nome da cidade: ")
            consulta_cidade = ConsultaCidadesMatereologia(nome_cidade)
            consulta_cidade.buscar_dados()

            while True:
                print("\n1 - Informações da Cidade")
                print("2 - Informações Meteorológicas")
                print("0 - Voltar")
                opcao = input("\nEscolha a opção: ")

                if opcao == '1':
                    try:
                        consulta_cidade.informacoes_cidade()
                    except Exception as e:
                        print(f"Ocorreu um erro ao consultar informações da cidade: {e}")
                        retry = input("Deseja tentar novamente? (s/n): ")
                        if retry.lower() != 's':
                            break

                elif opcao == '2':
                    try:
                        consulta_cidade.informacoes_metereologicas_cidade()
                    except Exception as e:
                        print(f"Ocorreu um erro ao consultar informações meteorológicas: {e}")
                        retry = input("Deseja tentar novamente? (s/n): ")
                        if retry.lower() != 's':
                            break

                elif opcao == '0':
                    break
                else:
                    print("Opção inválida!")
            break

        except Exception as e:
              print(f"Ocorreu um erro ao buscar os dados da cidade: {e}")
              retry = input("Deseja tentar novamente? (s/n): ")
              if retry.lower() != 's':
                  print("Saindo do menu de consulta de cidades.")
                  break



def main():

    db = banco

    while True:
        opcao = menu_principal()

        if opcao == '1':
            menu_empresa()
        elif opcao == '2':
            menu_cep()
        elif opcao == '3':
            menu_corretora()
        elif opcao == '4':
            menu_cidades()
        elif opcao == '5':
            menu_database(db)
        elif opcao == '0':
            print("Saindo do sistema...")
            break
        else:
            print("Opção inválida! Tente novamente.")


### Execução do programa

In [ ]:
if __name__ == "__main__":
    main()


Bem-vindo ao Sistema de Consulta!
Escolha uma das opções abaixo:
1 - Consulta de Dados de Empresa (CNPJ)
2 - Consulta de CEP
3 - Consulta de Corretora (CNPJ)
4 - Consulta de Cidades e Dados Meteorológicos
5 - Gerenciamento do Banco de Dados
0 - Sair

Digite a opção desejada: 3

Digite o CNPJ da corretora: 02332886000104

1 - Informações de Logradouro
2 - Informações de Cadastro
3 - Informações Patrimoniais
0 - Voltar

Escolha a opção: 2
DADOS CADASTRAIS DA CORRETORA
------------------------------------------------------------------------------------------------------------------------------------------------------
CNPJ:02332886000104  | Razão Social: XP INVESTIMENTOS CCTVM S.A. | Nome Fantasia: XP INVESTIMENTOS | Status: EM FUNCIONAMENTO NORMAL | Email: fabricio.almeida@xpi.com.br         | Telefone: 30272237

1 - Informações de Logradouro
2 - Informações de Cadastro
3 - Informações Patrimoniais
0 - Voltar

Escolha a opção: 1
DADOS DE LOGRADOURO DA CORRETORA
--------------------------